In [1]:
%%capture

!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq
!apt-get install maven -qq

!curl -L "https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz" > spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark py4j
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql import functions
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
#from pyspark.sql.Column import isNull
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "16g").getOrCreate()

In [2]:
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("CDC Pipeline").getOrCreate()


In [5]:

fldf=spark.read.csv("LOAD00000001.csv")
fldf=fldf.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","City")
fldf.write.mode("overwrite").csv("FinalOutput/finalFile.csv")


In [20]:
# Updated dataframe

udf=spark.read.csv("20240115-101454203.csv")
udf=udf.withColumnRenamed("_c0","action").withColumnRenamed("_c1","id").withColumnRenamed("_c2","FullName").withColumnRenamed("_c3","City")
udf.show()

udf2=spark.read.csv("20240115-101946772.csv")
udf2=udf2.withColumnRenamed("_c0","action").withColumnRenamed("_c1","id").withColumnRenamed("_c2","FullName").withColumnRenamed("_c3","City")
udf2.show()

# read full load data from here
ffdf=spark.read.csv("FinalOutput/finalFile.csv")
ffdf=ffdf.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","City")
ffdf.show()




+------+---+----------+--------+
|action| id|  FullName|    City|
+------+---+----------+--------+
|     U|  7|   ABC XYZ| Phoenix|
|     I|130|Alica Bing|New York|
+------+---+----------+--------+

+------+---+-------------+--------+
|action| id|     FullName|    City|
+------+---+-------------+--------+
|     I|131| Malinda Bing| Detroit|
|     I|132|Chandler Bing|Portland|
|     U|  8|      ABC XYZ|  Denver|
|     D| 10|   Jack Hicks| Houston|
+------+---+-------------+--------+

+---+----------------+-------------+
| id|        FullName|         City|
+---+----------------+-------------+
|  0|Herman Zimmerman|Oklahoma City|
|  1|        Lisa Ray|     Columbus|
|  2|  Terrell Reeves| Jacksonville|
|  3|   Steve Goodwin|    Charlotte|
|  4|       Leah Tran|      Detroit|
|  5|  Wilbert Holmes|   Washington|
|  6|    Mindy George|  Los Angeles|
|  7|       Rosa Huff|      Phoenix|
|  8|Clayton Jennings|       Denver|
|  9|     Darla Hayes|    Charlotte|
| 10|      Jack Hicks|      Hou

In [19]:
# actual CDC Change is happening here

for row in udf.collect():
  if(row["action"]== 'U'):
    ffdf=ffdf.withColumn("FullName",when(ffdf["id"]==row["id"],row["FullName"]).otherwise(ffdf["FullName"]))  # whereever row id matches we will update
    ffdf=ffdf.withColumn("City",when(ffdf["id"]==row["id"],row["City"]).otherwise(ffdf["City"]))  # whereever row id matches we will update


  if(row["action"]=='I'):
    insertRow= [list(row)[1:]]
    columns=['id','FullName','City']
    newdf=spark.createDataFrame(insertRow,columns)
    ffdf=ffdf.union(newdf)

  if(row["action"]=='D'):
    ffdf=ffdf.filter(ffdf.id!=row["id"])

for row in udf2.collect():
  if(row["action"]== 'U'):
    ffdf=ffdf.withColumn("FullName",when(ffdf["id"]==row["id"],row["FullName"]).otherwise(ffdf["FullName"]))  # whereever row id matches we will update
    ffdf=ffdf.withColumn("City",when(ffdf["id"]==row["id"],row["City"]).otherwise(ffdf["City"]))  # whereever row id matches we will update


  if(row["action"]=='I'):
    insertRow= [list(row)[1:]]
    columns=['id','FullName','City']
    newdf=spark.createDataFrame(insertRow,columns)
    ffdf=ffdf.union(newdf)

  if(row["action"]=='D'):
    ffdf=ffdf.filter(ffdf.id!=row["id"])


#ffdf.show()
ffdf.write.mode("overwrite").csv("FinalOutput/finalFile.csv")  # write output to file so after next batch we update latest data






Row(action='U', id='7', FullName='ABC XYZ', City='Phoenix')
Row(action='I', id='130', FullName='Alica Bing', City='New York')
Row(action='I', id='131', FullName='Malinda Bing', City='Detroit')
Row(action='I', id='132', FullName='Chandler Bing', City='Portland')
Row(action='U', id='8', FullName='ABC XYZ', City='Denver')
Row(action='D', id='10', FullName='Jack Hicks', City='Houston')
+---+----------------+-------------+
| id|        FullName|         City|
+---+----------------+-------------+
|  0|Herman Zimmerman|Oklahoma City|
|  1|        Lisa Ray|     Columbus|
|  2|  Terrell Reeves| Jacksonville|
|  3|   Steve Goodwin|    Charlotte|
|  4|       Leah Tran|      Detroit|
|  5|  Wilbert Holmes|   Washington|
|  6|    Mindy George|  Los Angeles|
|  7|         ABC XYZ|      Phoenix|
|  8|         ABC XYZ|       Denver|
|  9|     Darla Hayes|    Charlotte|
| 11|Francis Davidson|       Austin|
| 12|  Jerome Padilla|San Francisco|
| 13|    Mamie Duncan|      Houston|
| 14|      Julia Cain| 